In [1]:
## Full backend analysis for the CREEDS and L1000 dataset 
import os
import pandas as pd
import itertools
from pandas.compat import StringIO
import numpy as np
from numpy import loadtxt
import sys
import json
from pprint import pprint
import objectpath
import csv
import re
import matplotlib.pyplot as plt
import json, requests
from pprint import pprint
import scipy
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances
from clustergrammer_widget import *
def get_geneset(df, indexer):
    df_ = df.loc[indexer, :]
    return list(df_[df_ == 1].index)

In [2]:
## load in the pre-formed datasets from the L1000_Analysis and CREEDS_Analysis files.
## THIS TAKES A WHILE TO LOAD, SO ONLY LOAD THIS ONCE AND EARLY

# L1000 up and down gene loads for drug signatures
L1000_up_genes = pd.read_csv("L1000_up_genes.csv")
L1000_down_genes = pd.read_csv("L1000_down_genes.csv")

# CREEDS up and down genes for disease signatures
with open("disease_signatures-v1.0.json") as f:
    CREEDS_data = json.load(f)

# generate the up and down gene signatures
CREEDS_up_genes = {
    row['do_id']: row['up_genes']
    for row in CREEDS_data
}
CREEDS_down_genes = {
    row['do_id']: row['down_genes']
    for row in CREEDS_data
}

# load in the EMR Data (filtered > 200 in R code [Drug_diagnosis_test_code.R])
EMR_data = pd.read_csv("EMR_greater_200.csv")
## subset EMR data by the DOI and/or DrOI
EMR_data_df = pd.DataFrame(EMR_data)
#EMR_data
EMR_data_df.drop(EMR_data_df.columns[[0]], axis = 1, inplace = True) # remove the unecessary columns
#EMR_data_df

# implement the search from ICD9-do_id from the manual conversion
icd9_to_doid = pd.read_csv("ICD9_CREEDS_conversion.csv")
icd9_to_doid = pd.DataFrame(icd9_to_doid) # convert it to a data fram to drop unecessary rows
#icd9_to_doid # sanity check
icd9_to_doid_final = icd9_to_doid.drop(icd9_to_doid.columns[[0, 6, 7, 8, 9, 10, 11, 12, 13, 14]], axis = 1)
#icd9_to_doid_final # sanity check

## PRELOADED DATA
## L1000 conversion and analysis DRUG
L1000_down_extract = pd.DataFrame(L1000_down_genes)
L1000_up_extract = pd.DataFrame(L1000_up_genes)

## L1000 ANALYSIS -- FEEDS BACK INTO THE API to get additional signatures
metadata = pd.read_csv("L1000_metadata.csv")
#metadata ## same as LINC1000h5.row_metadata_df

EMR_Drug_Names = EMR_data_df['Drug_Name'] # this will be the selection for the dropdown menu
## get the subselection of drug names

unique_pert_ids = metadata['pert_id'].unique()
unique_drug_names = metadata['pert_desc'].unique()

possible_drug_inputs = set(unique_drug_names) & set(EMR_Drug_Names) # 28 possible drug inputs


In [6]:
## X2K API integration 

# Import modules
import http.client
import json

##### Function to run X2K
### Input: a Python list of gene symbols
### Output: a dictionary containing the results of X2K, ChEA, G2N, KEA.


def run_X2K(input_genes, options={}):
    # Open HTTP connection
    conn = http.client.HTTPConnection("amp.pharm.mssm.edu")

    # Set default options
    default_options = {'text-genes': '\n'.join(input_genes),
                       'included_organisms': 'both',
                       'TF-target gene background database used for enrichment': 'ChEA & ENCODE Consensus',
                       'sort transcription factors by': 'p-value',
                       'min_network_size': 10,
                       'number of top TFs': 10,
                       'path_length': 2,
                       'min_number_of_articles_supporting_interaction': 0,
                       'max_number_of_interactions_per_protein': 200,
                       'max_number_of_interactions_per_article': 100,
                       'enable_BioGRID': True,
                       'enable_IntAct': True,
                       'enable_MINT': True,
                       'enable_ppid': True,
                       'enable_Stelzl': True,
                       'kinase interactions to include': 'kea 2018',
                       'sort kinases by': 'p-value'}

    # Update options
    for key, value in options.items():
        if key in default_options.keys() and key != 'text-genes':
            default_options.update({key: value})

    # Get payload
    boundary = "----WebKitFormBoundary7MA4YWxkTrZu0gW"
    payload = ''.join(
        ['--' + boundary + '\r\nContent-Disposition: form-data; name=\"{key}\"\r\n\r\n{value}\r\n'.format(**locals())
         for key, value in default_options.items()]) + '--' + boundary + '--'

    # Get Headers
    headers = {
        'content-type': "multipart/form-data; boundary=" + boundary,
        'cache-control': "no-cache",
    }

    # Initialize connection
    conn.request("POST", "/X2K/api", payload, headers)

    # Get response
    res = conn.getresponse()

    # Read response
    data = res.read().decode('utf-8')

    # Convert to dictionary
    x2k_results = {key: json.loads(value) if key != 'input' else value for key, value in json.loads(data).items()}

    # Clean results
    x2k_results['ChEA'] = x2k_results['ChEA']['tfs']
    x2k_results['G2N'] = x2k_results['G2N']['network']
    x2k_results['KEA'] = x2k_results['KEA']['kinases']
    x2k_results['X2K'] = x2k_results['X2K']['network']

    # Return results
    return x2k_results
####################################################################################################################################

### try the input
DrOI = "DIGOXIN" # drug of interest. MAKE SURE TO SELECT THIS FROM "possible_drug_inputs"
####

DrOI_up_extract = L1000_up_extract[L1000_up_extract['Unnamed: 0'].apply(lambda s: bool(re.compile(DrOI, re.IGNORECASE).search(s)))]
DrOI_up_final = DrOI_up_extract.loc[:, (DrOI_up_extract != 0).any(axis=0)] # remove any genes without any expression in any of the results

DrOI_down_extract = L1000_down_extract[L1000_down_extract['Unnamed: 0'].apply(lambda s: bool(re.compile(DrOI, re.IGNORECASE).search(s)))]
DrOI_down_final = DrOI_down_extract.loc[:, (DrOI_down_extract != 0).any(axis=0)] # remove any genes without any expression in any of the results



### CREEDS DRUG CARD 


#for a in loop_iteration:
CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
CREEEDS_Drug_response = requests.get(CREEDS_URL + 'search', params={'q':DrOI})
if CREEEDS_Drug_response.status_code == 200:
    #pprint(CREEEDS_Drug_response.json())
    #json.dump(CREEEDS_Drug_response.json(), open(DrOI + '_api1_result.json', 'w'), indent=4)
    CREEDS_drug_output_df = pd.DataFrame(CREEEDS_Drug_response.json())

#CREEDS_drug_output_df #Display this table
CREEDS_drug_sigs = list(CREEDS_drug_output_df.id)

for a in CREEDS_drug_sigs:
    CREEDS_X2K_drug_response = requests.get(CREEDS_URL + 'api', params={'id':str(a)})
    if CREEDS_X2K_drug_response.status_code == 200:
        #pprint(CREEDS_X2K_drug_response.json())
        #json.dump(CREEDS_X2K_drug_response.json(), open('api3_result.json', 'wb'), indent=4)
        CREEDS_signatures_total = CREEDS_X2K_drug_response.json()
        
        
        CREEDS_query_up_genes = CREEDS_signatures_total["up_genes"]
        CREEDS_query_up_genes = pd.DataFrame(CREEDS_query_up_genes)
        CREEDS_query_up_genes.columns = ["Up_genes", "Score"]
        CREEDS_query_up_genes = list(CREEDS_query_up_genes["Up_genes"])
        
        CREEDS_query_down_genes = CREEDS_signatures_total["down_genes"]
        CREEDS_query_down_genes = pd.DataFrame(CREEDS_query_down_genes)
        CREEDS_query_down_genes.columns = ["Down_genes", "Score"]
        CREEDS_query_down_genes = list(CREEDS_query_down_genes["Down_genes"])
        
        ## need to clean the creeds genes because they have p values attached
        
        
        ## X2K implementation
        
        ## up genes
        CREEDS_X2K_up_genes = run_X2K(CREEDS_query_up_genes)
        CREEDS_X2K_up_genes = CREEDS_X2K_up_genes["X2K"]
        CREEDS_X2K_up_genes_df = pd.DataFrame(CREEDS_X2K_up_genes['nodes'])
        filename_up = (a + "_CREEDS_X2K_up_genes.csv")
        CREEDS_X2K_up_genes_df.to_csv(filename_up) # THIS IS THE FILE THEY SHOULD BE ABLE TO DOWNLOAD
        
        ## down genes
        CREEDS_X2K_down_genes = run_X2K(CREEDS_query_down_genes)
        CREEDS_X2K_down_genes = CREEDS_X2K_down_genes["X2K"]
        CREEDS_X2K_down_genes_df = pd.DataFrame(CREEDS_X2K_down_genes['nodes'])
        filename_down = (a + "_CREEDS_X2K_down_genes.csv")
        CREEDS_X2K_down_genes_df.to_csv(filename_down) # THIS IS THE FILE THEY SHOULD BE ABLE TO DOWNLOAD
        
    else:
        print("No significant CREEDS drug signatures found!")